[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fisamz/Repositorio_MCDAA/blob/main/Tarea2/Tarea2.ipynb)

# Tarea 2 — RDD en PySpark
**Alumno:** Fisam Zavala  
**Dataset:** Resultados de futbol & momios de casas de apuestas.  
**Fuente:** [European Soccer Database](https://www.kaggle.com/datasets/hugomathien/soccer)


In [1]:
#%pip install pyspark
import pyspark
pyspark.__version__

'4.1.1'

## Punto 1. Origen de datos (adquisición)

Para esta práctica se utiliza el conjunto de datos **European Soccer Database** (Kaggle), el cual contiene información histórica de ligas europeas: países, ligas, equipos, jugadores y resultados de partidos.

El dataset se proporciona en formato **SQLite** (`database.sqlite`), lo cual representa un origen de datos estructurado y relacional.  
A partir de este archivo se extraen las tablas relevantes (por ejemplo `Match` y `Team`) y se exportan a archivos `.csv` para facilitar su posterior procesamiento con PySpark.

In [ ]:
# (Opcional / referencia) Descarga desde Kaggle:
# !pip install kaggle
# !kaggle datasets download -d hugomathien/soccer
# !unzip soccer.zip -d data/

#import sqlite3, pandas as pd

#conn = sqlite3.connect("../data/database.sqlite")

#df_match = pd.read_sql("SELECT * FROM Match", conn)
#df_match.to_csv("../data/match.csv", index=False)

#conn.close()


## Punto 2. Conversión del origen de datos a RDD con PySpark

A partir del archivo `match.csv`, se utilizó `SparkContext.textFile()` para cargar la información en forma de un RDD, donde cada elemento representa una línea del archivo.

Posteriormente, se eliminó el encabezado y se aplicó un proceso de parseo robusto mediante la librería `csv`, con el fin de separar correctamente las columnas y evitar errores ocasionados por filas inconsistentes.

Las filas inválidas o incompletas fueron descartadas mediante filtros, conservando únicamente los registros con el número correcto de columnas. A partir de esta información, se seleccionaron las variables relevantes: temporada, fecha y goles de los equipos local y visitante.

Finalmente, se construyó un RDD con tuplas del tipo:

(season, date, home_goals, away_goals, total_goals)

donde `total_goals` corresponde a la suma de goles anotados en cada partido. Este RDD fue almacenado en memoria mediante `cache()` para optimizar su reutilización en análisis posteriores.


In [4]:
from pyspark.sql import SparkSession
import csv
from io import StringIO

spark = SparkSession.builder.appName("Tarea2_RDD").master("local[*]").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
#sc

# 1) Leer como texto (esto ya es un RDD)
rdd_text = sc.textFile("../data/match.csv")

# Ver primeras líneas
rdd_text.take(3)


['id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4

In [12]:
import csv

# 1) Leer archivo como RDD
rdd_text = sc.textFile("../data/match.csv")

# 2) Obtener y limpiar header
header = rdd_text.first()
rdd_no_header = rdd_text.filter(lambda row: row.strip() != header.strip())

# 3) Parser seguro
def parse_csv_line_safe(line: str):
    try:
        return next(csv.reader([line]))
    except:
        return None

rdd_parsed = rdd_no_header.map(parse_csv_line_safe).filter(lambda x: x is not None)

# 4) Validar longitud correcta
cols = next(csv.reader([header]))
expected_len = len(cols)

rdd_clean = rdd_parsed.filter(lambda r: len(r) == expected_len)

# 5) Índices de columnas importantes
idx_date   = cols.index("date")
idx_home   = cols.index("home_team_goal")
idx_away   = cols.index("away_team_goal")
idx_season = cols.index("season")

def safe_int(x):
    try:
        return int(float(x))
    except:
        return None

# 6) Construir RDD final
rdd_match_basic = rdd_clean.map(lambda r: (
    r[idx_season],
    r[idx_date],
    safe_int(r[idx_home]),
    safe_int(r[idx_away]),
)).map(lambda t: (t[0], t[1], t[2], t[3], (t[2] or 0) + (t[3] or 0)))

# 7) Cache (opcional pero recomendado)
rdd_match_basic = rdd_match_basic.cache()



In [13]:
print("Registros en RDD:", rdd_match_basic.count())
print("Particiones:", rdd_match_basic.getNumPartitions())

Registros en RDD: 25978
Particiones: 9


In [14]:
rdd_match_basic.take(5)

[('2008/2009', '2008-08-17 00:00:00', 1, 1, 2),
 ('2008/2009', '2008-08-16 00:00:00', 0, 0, 0),
 ('2008/2009', '2008-08-16 00:00:00', 0, 3, 3),
 ('2008/2009', '2008-08-17 00:00:00', 5, 0, 5),
 ('2008/2009', '2008-08-16 00:00:00', 1, 3, 4)]

## Punto 3. Operaciones y estadísticas descriptivas en el RDD

Con el RDD `rdd_match_basic` se realizaron operaciones básicas para explorar la distribución de goles y resultados:

- Se calcularon estadísticas descriptivas del total de goles por partido (conteo, promedio, mínimo y máximo).
- Se comparó el promedio de goles del equipo local vs visitante.
- Se evaluó la frecuencia de partidos con alta anotación (≥ 5 goles).
- Se estimó la ventaja de local calculando el porcentaje de victorias del local, empates y victorias del visitante.
- Se agregaron métricas por temporada utilizando RDDs tipo (key, value) y `reduceByKey`, obteniendo el promedio de goles por temporada.

Estos cálculos muestran el uso de transformaciones (`map`, `filter`, `reduceByKey`) y acciones (`count`, `sum`, `min`, `max`, `takeOrdered`) propias de RDD en Spark.


In [15]:
# RDD con total_goals solamente (ignoramos None por seguridad)
rdd_total_goals = rdd_match_basic.map(lambda x: x[4]).filter(lambda g: g is not None)

n = rdd_total_goals.count()
sum_goals = rdd_total_goals.sum()
mean_goals = sum_goals / n if n > 0 else None

min_goals = rdd_total_goals.min()
max_goals = rdd_total_goals.max()

print("N partidos:", n)
print("Promedio total de goles:", round(mean_goals, 4))
print("Mínimo total de goles:", min_goals)
print("Máximo total de goles:", max_goals)


N partidos: 25978
Promedio total de goles: 2.7056
Mínimo total de goles: 0
Máximo total de goles: 12


In [16]:
rdd_home_goals = rdd_match_basic.map(lambda x: x[2]).filter(lambda g: g is not None)
rdd_away_goals = rdd_match_basic.map(lambda x: x[3]).filter(lambda g: g is not None)

n_home = rdd_home_goals.count()
n_away = rdd_away_goals.count()

mean_home = rdd_home_goals.sum() / n_home if n_home > 0 else None
mean_away = rdd_away_goals.sum() / n_away if n_away > 0 else None

print("Promedio goles equipo local:", round(mean_home, 4))
print("Promedio goles equipo visitante:", round(mean_away, 4))


Promedio goles equipo local: 1.5446
Promedio goles equipo visitante: 1.161


In [17]:
count_5plus = rdd_total_goals.filter(lambda g: g >= 5).count()
pct_5plus = (count_5plus / n) * 100 if n > 0 else None

print("Partidos con >= 5 goles:", count_5plus)
print("% partidos con >= 5 goles:", round(pct_5plus, 2))


Partidos con >= 5 goles: 3629
% partidos con >= 5 goles: 13.97


In [18]:
# (home_goals, away_goals) válidos
rdd_results = rdd_match_basic.map(lambda x: (x[2], x[3])) \
                             .filter(lambda t: (t[0] is not None) and (t[1] is not None))

n_res = rdd_results.count()

home_wins = rdd_results.filter(lambda t: t[0] > t[1]).count()
draws = rdd_results.filter(lambda t: t[0] == t[1]).count()
away_wins = rdd_results.filter(lambda t: t[0] < t[1]).count()

print("Total partidos evaluados:", n_res)
print("Home wins:", home_wins, "| %:", round(home_wins/n_res*100, 2))
print("Draws:", draws, "| %:", round(draws/n_res*100, 2))
print("Away wins:", away_wins, "| %:", round(away_wins/n_res*100, 2))


Total partidos evaluados: 25978
Home wins: 11916 | %: 45.87
Draws: 6596 | %: 25.39
Away wins: 7466 | %: 28.74


In [19]:
# (season, (total_goals, 1))
rdd_season_pairs = rdd_match_basic \
    .filter(lambda x: x[0] is not None and x[4] is not None) \
    .map(lambda x: (x[0], (x[4], 1)))

# sumas por temporada: (season, (sum_goals, count))
rdd_season_agg = rdd_season_pairs.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# promedio: (season, avg_goals, num_matches)
rdd_season_avg = rdd_season_agg.map(lambda kv: (kv[0], kv[1][0] / kv[1][1], kv[1][1]))

# Top 10 temporadas con más promedio de goles
top10 = rdd_season_avg.takeOrdered(10, key=lambda x: -x[1])

for s, avg, cnt in top10:
    print(s, "| avg_goals:", round(avg, 3), "| matches:", cnt)


2012/2013 | avg_goals: 2.773 | matches: 3260
2013/2014 | avg_goals: 2.767 | matches: 3031
2015/2016 | avg_goals: 2.755 | matches: 3326
2011/2012 | avg_goals: 2.716 | matches: 3220
2010/2011 | avg_goals: 2.684 | matches: 3260
2014/2015 | avg_goals: 2.676 | matches: 3325
2009/2010 | avg_goals: 2.672 | matches: 3230
2008/2009 | avg_goals: 2.607 | matches: 3326
